In [11]:
import requests
import pandas as pd
import urllib.request
import json

AccountKey=""

datasets = {'bus_routes': "http://datamall2.mytransport.sg/ltaodataservice/BusRoutes",
            'bus_services': "http://datamall2.mytransport.sg/ltaodataservice/BusServices",
            'bus_stops': "http://datamall2.mytransport.sg/ltaodataservice/BusStops",
            'bus_arrival': "http://datamall2.mytransport.sg/ltaodataservice/BusArrivalv2",
            'taxi_availability': "http://datamall2.mytransport.sg/ltaodataservice/Taxi-Availability",
            'taxi_stands': "http://datamall2.mytransport.sg/ltaodataservice/TaxiStands",
            'train_service_alers': "http://datamall2.mytransport.sg/ltaodataservice/TrainServiceAlerts",
            'carpark_availability': "http://datamall2.mytransport.sg/ltaodataservice/CarParkAvailabilityv2",
            'erp_rates': "http://datamall2.mytransport.sg/ltaodataservice/ERPRates",
            'est_travel_times': "http://datamall2.mytransport.sg/ltaodataservice/EstTravelTimes",
            'faulty_traffic_lights': "http://datamall2.mytransport.sg/ltaodataservice/FaultyTrafficLights",
            'road_openings': "http://datamall2.mytransport.sg/ltaodataservice/RoadOpenings", 
            'road_works': "http://datamall2.mytransport.sg/ltaodataservice/RoadWorks", 
            'traffic_images': "http://datamall2.mytransport.sg/ltaodataservice/Traffic-Imagesv2",
            'traffic_incidents': "http://datamall2.mytransport.sg/ltaodataservice/TrafficIncidents",
            'traffic_speed_bands': "http://datamall2.mytransport.sg/ltaodataservice/v3/TrafficSpeedBands",
            'vms_emas': "http://datamall2.mytransport.sg/ltaodataservice/VMS",
            'bicycle_parking': "http://datamall2.mytransport.sg/ltaodataservice/BicycleParkingv2",
            'platform_crowd_density': "http://datamall2.mytransport.sg/ltaodataservice/PCDRealTime",
            'platform_crowd_density_forecase': "http://datamall2.mytransport.sg/ltaodataservice/PCDForecast", 
            'traffic_flow': "http://datamall2.mytransport.sg/ltaodataservice/TrafficFlow",
            'facilities_maintanance': "http://datamall2.mytransport.sg/ltaodataservice/FacilitiesMaintenance",
            'geospatial_island_map': "http://datamall2.mytransport.sg/ltaodataservice/GeospatialWholeIslan", 
            'passenger_vol_by_train_station': "http://datamall2.mytransport.sg/ltaodataservice/PV/Train",
            'passenger_vol_by_origin_train_station': "http://datamall2.mytransport.sg/ltaodataservice/PV/ODTrain",
            'passenger_vol_by_origin_bus_stop': "http://datamall2.mytransport.sg/ltaodataservice/PV/ODBus",
            'passenger_vol_by_bus_stop': "http://datamall2.mytransport.sg/ltaodataservice/PV/Bus",
            
            'faulty_traffic_lights': "http://datamall2.mytransport.sg/ltaodataservice/FaultyTrafficLights",
            'traffic_incidents': "http://datamall2.mytransport.sg/ltaodataservice/TrafficIncidents",
            'road_openings': "http://datamall2.mytransport.sg/ltaodataservice/RoadOpenings", 
            'road_works': "http://datamall2.mytransport.sg/ltaodataservice/RoadWorks",
            'traffic_flow': "http://datamall2.mytransport.sg/ltaodataservice/TrafficFlow", 
            'estimated_travel_times': "http://datamall2.mytransport.sg/ltaodataservice/EstTravelTimes",
            'carpark_avail': "http://datamall2.mytransport.sg/ltaodataservice/CarParkAvailabilityv2",
            'traffic_flow': "http://datamall2.mytransport.sg/ltaodataservice/TrafficFlow"
            }

payload = {}

# API key
headers = {
  'AccountKey': AccountKey
}

In [12]:
dataset_with_link = "traffic_flow"

In [13]:
import time
import pandas as pd
import os
from copy import deepcopy
import numpy as np
from datetime import datetime
import re

#This makes calls to all datasets
def obtain_data(data_source_name):
    df = {}
    for i, v in datasets.items():
        response = requests.request("GET", v, headers=headers, data=payload)

        if response.status_code == 200:
            data = response.json()

            if 'value' in data:
                df[i] = pd.DataFrame(data['value'])

    return df[data_source_name]

#this makes call to one dataset at a time
def obtain_data_per_dataset(data_source_name):
    response = requests.get(datasets[data_source_name], headers=headers,data=payload)
    if response.status_code == 200:
        data = response.json()
        
        if 'value' in data:
            df = pd.DataFrame(data['value'])
            return df
    else:
        response.raise_for_status()

def append_new_data(source_path, source_filename, curr_df, drop_dup_by_fields, printout=False):
    # If the source directory doesn't exist, create it
    if not os.path.exists(source_path):
        os.makedirs(source_path)

    source_file_path = os.path.join(source_path, source_filename)

    # Check if the file exists
    if os.path.exists(source_file_path):
        try:
            source_df = pd.read_csv(source_file_path)
            
        except pd.errors.EmptyDataError:
            print(f"Warning: {source_file_path} is empty.")
            source_df = pd.DataFrame()  
    else:
        source_df = pd.DataFrame()
        print(f"No existing data found, creating a new data file.")

    # Identify new data
    if not source_df.empty:
        if len(drop_dup_by_fields) > 0:
            df = pd.concat([curr_df, source_df], ignore_index=True).drop_duplicates(subset=drop_dup_by_fields)
        else:
            df = pd.concat([curr_df, source_df], ignore_index=True).drop_duplicates()
    else:
        df = curr_df 
        
    df.to_csv(source_file_path, index=False)
    msg = f"Saved {len(df)} records to path: {source_file_path}"
    if printout: print(msg)
    
    
def obtain_and_append(api_file_name, save_data_to, source_file_name, drop_dup_by_fields, printout):
    
    if api_file_name == "traffic_incidents":
        #df = obtain_data(api_file_name)
        df = obtain_data_per_dataset(api_file_name) #changed the function here to newer version
        df['Date_time'] = df['Message'].apply(lambda x: format_datetime(x))
    if api_file_name == 'traffic_flow':
        df = obtain_data_from_link(dataset_with_link) #added this if dataset has a link
    else:
        df = obtain_data(api_file_name)
        
    append_new_data(
        source_path=save_data_to, 
        source_filename=source_file_name, 
        curr_df=df,
        drop_dup_by_fields = drop_dup_by_fields,
        printout = printout
    )

def format_datetime(datetime_str):
    match = re.search(r'\((\d{1,2}/\d{1,2})\)(\d{1,2}:\d{2})', datetime_str)
    if match:
        extracted_date = match.group(1)
        extracted_time = match.group(2)
        current_year = datetime.now().year
        extracted_date_parts = extracted_date.split('/')
        formatted_date = f"{current_year}-{extracted_date_parts[1].zfill(2)}-{extracted_date_parts[0].zfill(2)}"
        formatted_datetime = f"{formatted_date}-{extracted_time}"
        return formatted_datetime
    else:
        return datetime_str


def obtain_data_from_link(dataset_with_link):
    link = obtain_data_per_dataset(dataset_with_link)  # Make sure obtain_data() returns the correct information
    access_here = link['Link'].iloc[0]
    print(access_here)

    with urllib.request.urlopen(access_here) as response:
        contents = response.read()
        try:
            text = contents.decode('utf-8')
        except UnicodeDecodeError:
            
            text = contents.decode('iso-8859-1')
        
        json_data = json.loads(text)

    if 'Value' in json_data:
        df = pd.DataFrame(json_data['Value'])
        return df
    else:
        return None 

def generate_create_table_statement(df, table_name):
    sql = f"CREATE TABLE {table_name} (\n"
    
    column_definitions = []
    for column, dtype in df.dtypes.items():
        column_type = 'varchar(255)' 
        if pd.api.types.is_integer_dtype(dtype):
            column_type = 'int'
        elif pd.api.types.is_float_dtype(dtype):
            column_type = 'real'
        elif pd.api.types.is_datetime64_any_dtype(dtype):
            column_type = 'datetime'
        elif pd.api.types.is_string_dtype(dtype):
            column_type = 'varchar(255)'
        
        column_definitions.append(f"    {column} {column_type}")
    
    sql += ",\n".join(column_definitions)
    sql += "\n);"
    
    return sql


In [14]:
i = 0
while i < 10000:    
    obtain_and_append(
        api_file_name = "traffic_incidents", 
        save_data_to = "./Data/traffic_incidents", 
        source_file_name = "traffic_incidents.csv", 
        drop_dup_by_fields = ["Message"],
        printout=True
    )
    
    obtain_and_append(
        api_file_name = "faulty_traffic_lights", 
        save_data_to = "./Data/faulty_traffic_lights", 
        source_file_name = "faulty_traffic_lights.csv", 
        drop_dup_by_fields = ["AlarmID", "NodeID"],
        printout=True
    )
        
    obtain_and_append(
        api_file_name = "road_openings", 
        save_data_to = "./Data/road_openings", 
        source_file_name = "road_openings.csv", 
        drop_dup_by_fields = ["EventID"],
        printout=True
    )
        
    obtain_and_append(
        api_file_name = "road_works", 
        save_data_to = "./Data/road_works", 
        source_file_name = "road_works.csv", 
        drop_dup_by_fields = ["EventID"],
        printout=True
    )
        
    obtain_and_append(
        api_file_name = "estimated_travel_times", 
        save_data_to = "./Data/estimated_travel_times", 
        source_file_name = "estimated_travel_times.csv", 
        drop_dup_by_fields = ["Name", "Direction", "FarEndPoint", "StartPoint", "EndPoint", "EstTime"],
        printout=True
    )
        
    obtain_and_append(
        api_file_name = "carpark_avail", 
        save_data_to = "./Data/carpark_avail", 
        source_file_name = "carpark_avail.csv", 
        drop_dup_by_fields = ['CarParkID'],
        printout=True
    )

    obtain_and_append(
        api_file_name = "bus_routes", 
        save_data_to = "./Data/bus_routes", 
        source_file_name = "bus_routes.csv", 
        drop_dup_by_fields = ['ServiceNo', 'BusStopCode'],
        printout=True
    )

    obtain_and_append(
        api_file_name = "bus_stops", 
        save_data_to = "./Data/bus_stops", 
        source_file_name = "bus_stops.csv", 
        drop_dup_by_fields = ['BusStopCode'],
        printout=True
    )

    obtain_and_append(
        api_file_name = "taxi_availability", 
        save_data_to = "./Data/taxi_availability", 
        source_file_name = "taxi_availability.csv", 
        drop_dup_by_fields = [],
        printout=True
    )
    
    # obtain_and_append(
    #     api_file_name = "taxi_stands", 
    #     save_data_to = "./Data/taxi_stands", 
    #     source_file_name = "taxi_stands.csv", 
    #     drop_dup_by_fields = [],
    #     printout=True
    # )

    obtain_and_append(
        api_file_name = "traffic_speed_bands", 
        save_data_to = "./Data/traffic_speed_bands", 
        source_file_name = "traffic_speed_bands.csv", 
        drop_dup_by_fields = ['LinkID'],
        printout=True
    )

    # obtain_and_append(
    #     api_file_name = "traffic_flow", 
    #     save_data_to = "/Users/emmy/Desktop/Last_term_SMU/NUS_proj/nus_comp/data", 
    #     source_file_name = "traffic_flow.csv", 
    #     drop_dup_by_fields = None,
    #     printout=True
    # )
    
        
    print("\n")
    i += 1
    time.sleep(3)

Saved 121 records to path: ./Data/traffic_incidents\traffic_incidents.csv
Saved 0 records to path: ./Data/faulty_traffic_lights\faulty_traffic_lights.csv
Saved 8 records to path: ./Data/road_openings\road_openings.csv
Saved 233 records to path: ./Data/road_works\road_works.csv
Saved 200 records to path: ./Data/estimated_travel_times\estimated_travel_times.csv
Saved 411 records to path: ./Data/carpark_avail\carpark_avail.csv
Saved 984 records to path: ./Data/bus_routes\bus_routes.csv
Saved 1000 records to path: ./Data/bus_stops\bus_stops.csv
Saved 2438 records to path: ./Data/taxi_availability\taxi_availability.csv
Saved 1000 records to path: ./Data/traffic_speed_bands\traffic_speed_bands.csv


Saved 121 records to path: ./Data/traffic_incidents\traffic_incidents.csv
Saved 0 records to path: ./Data/faulty_traffic_lights\faulty_traffic_lights.csv
Saved 8 records to path: ./Data/road_openings\road_openings.csv
Saved 233 records to path: ./Data/road_works\road_works.csv
Saved 200 records t

In [ ]:
dataset_with_link = "erp_rates"
tmp_df = obtain_data_per_dataset(dataset_with_link)
tmp_df

#df_with_link = obtain_data_from_link(dataset_with_link)
#df_with_link

,VehicleType,DayType,StartTime,EndTime,ZoneID,ChargeAmount,EffectiveDate
0,Passenger Cars/Light Goods Vehicles/Taxis,Weekdays,07:00,08:00,AY1,0.0,2024-01-02
1,Passenger Cars/Light Goods Vehicles/Taxis,Weekdays,08:00,08:05,AY1,0.5,2024-01-02
2,Passenger Cars/Light Goods Vehicles/Taxis,Weekdays,08:05,08:30,AY1,1.0,2024-01-02
3,Passenger Cars/Light Goods Vehicles/Taxis,Weekdays,08:30,08:35,AY1,1.5,2024-01-02
4,Passenger Cars/Light Goods Vehicles/Taxis,Weekdays,08:35,08:55,AY1,2.0,2024-01-02
...,...,...,...,...,...,...,...
495,Light Goods Vehicles,Weekdays,17:30,18:00,CBD,0.0,2024-01-02
496,Light Goods Vehicles,Weekdays,18:00,18:05,CBD,0.0,2024-01-02
497,Light Goods Vehicles,Weekdays,18:05,18:25,CBD,0.0,2024-01-02
498,Light Goods Vehicles,Weekdays,18:25,18:30,CBD,0.0,2024-01-02


In [ ]:
print(generate_create_table_statement(tmp_df, dataset_with_link))

CREATE TABLE traffic_flow (
    LinkID varchar(255),
    Date varchar(255),
    HourOfDate varchar(255),
    Volume varchar(255),
    StartLon varchar(255),
    StartLat varchar(255),
    EndLon varchar(255),
    EndLat varchar(255),
    RoadName varchar(255),
    RoadCat varchar(255)
);
